In [1]:
import pandas as pd 
import numpy as np

In [2]:
# testing
print(f'jello {pd.__version__}')

jello 2.2.2


In [ ]:
# LC 184 - https://leetcode.com/problems/department-highest-salary/description/

"""
Employee table:
+----+-------+--------+--------------+
| id | name  | salary | departmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
Department table:
+----+-------+
| id | name  |
+----+-------+
| 1  | IT    |
| 2  | Sales |
+----+-------+


T-SQL :
WITH max_sal_cte AS (
    SELECT 
        e.name AS Employee,
        salary AS Salary, 
        d.name AS Department,
        ROW_NUMBER() OVER(PARTITION BY departmentId ORDER BY salary DESC) AS rn
    FROM Employee AS e
    INNER JOIN Department AS d       # left also works
    ON e.departmentId = d.id
)
SELECT Employee, Salary, Department 
FROM max_sal_cte
WHERE rn = 1;
"""


employee_data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
    'salary': [70000, 90000, 80000, 60000, 90000],
    'departmentId': [1, 1, 2, 2, 1]
}

department_data = {
    'id': [1, 2],
    'name': ['IT', 'Sales']
}

employee_df = pd.DataFrame(employee_data)
department_df = pd.DataFrame(department_data)


def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merged_df = employee.merge(department, left_on='departmentId', right_on='id', how='inner', suffixes=['_emp', '_dept'])
    print('merged_df \n', merged_df)
    max_salaries = merged_df.groupby('name_dept')['salary'].transform('max')
    print(max_salaries)
    result_df = merged_df[merged_df['salary'] == max_salaries]
    print(result_df)
    final_df = result_df[['name_dept', 'name_emp', 'salary']]
    final_df.columns = ['Departmen', 'Employee', 'Salary']
    
    return final_df


print(department_highest_salary(employee_df, department_df))

merged_df 
    id_emp name_emp  salary  departmentId  id_dept name_dept
0       1      Joe   70000             1        1        IT
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
3       4      Sam   60000             2        2     Sales
4       5      Max   90000             1        1        IT
0    90000
1    90000
2    80000
3    80000
4    90000
Name: salary, dtype: int64
   id_emp name_emp  salary  departmentId  id_dept name_dept
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
4       5      Max   90000             1        1        IT
  Departmen Employee  Salary
1        IT      Jim   90000
2     Sales    Henry   80000
4        IT      Max   90000


In [ ]:
# LC 117 - https://leetcode.com/problems/nth-highest-salary/description/

"""
T SQL

CREATE FUNCTION dbo.getNthHighestSalary (@N INT)
RETURNS INT
AS
BEGIN
    -- Return NULL if N is invalid
    IF @N < 1
        RETURN NULL;

    RETURN (
        SELECT DISTINCT(salary)
        FROM Employee
        ORDER BY salary DESC
        OFFSET (@N - 1) ROWS FETCH NEXT 1 ROWS ONLY
    );
END;
"""


data = [[1, 100], [2, 200], [3, 300]]
employee_df = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

def nth_highest_salary(employee: pd.DataFrame, n: int) -> pd.DataFrame:
    sorted_salary = employee['salary'].sort_values(ascending=False)
    unique_values = sorted_salary.unique()
    
    col_name = f'getNthHighestSalary({n})'
    
    if n < 1 or n > len(unique_values):
        return pd.DataFrame({col_name: [None]})
    else:
        nth_val = int(unique_values[n - 1])
        return pd.DataFrame({col_name: [nth_val]})


nth_highest_salary(employee_df, 2)

200


200

In [13]:
# LC 570 - https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/


"""
TSQL
Q1
-- select m.name
-- from employee m
-- join employee e
-- on m.id = e.managerId
-- group by m.id, m.name
-- having count(e.managerId) >= 5;

Q2
select m.name as name
from employee m
join (
    select managerId 
    from employee 
    group by managerId
    having count(managerId) >= 5
) as e
on m.id = e.managerId

here, Q2 is faster because it filters the managerId first
and then joins. this gives improved performance on large table 
"""


data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
emp_df = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    df = employee.groupby('managerId')['id'].count().reset_index().rename(columns={'id': 'count'})
    five_reports = df[df['count'] >= 5]['managerId'].values
    return employee[employee['id'].isin(five_reports)][['name']]

print(find_managers(emp_df))

   name
0  John


In [ ]:
# LC 1934 - https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50

"""
T SQL

SELECT 
    s.user_id,
    ROUND(
        CASE
            WHEN COUNT(c.action) = 0 THEN 0
            ELSE SUM(CASE WHEN c.action = 'confirmed' THEN 1 ELSE 0 END) * 1.0 / COUNT(c.action)
        END 
    , 2) AS confirmation_rate 
FROM Signups AS s
LEFT JOIN Confirmations AS c
ON s.user_id = c.user_id
GROUP BY s.user_id

"""


data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups_df = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations_df = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})


def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:
    combined_df = pd.merge(signups, confirmations, how='left', on='user_id')
    combined_df['action_code'] = combined_df['action'].map({'confirmed': 1, 'timeout': 0})
    combined_df = combined_df.groupby('user_id')['action_code'].mean().round(2).fillna(0).reset_index()
    combined_df.columns = ['user_id', 'confirmation_rate']
    return combined_df


confirmation_rate(signups_df, confirmations_df)

   user_id        time_stamp_x        time_stamp_y     action  action_code
0        3 2020-03-21 10:16:13 2021-01-06 03:30:46    timeout          0.0
1        3 2020-03-21 10:16:13 2021-07-14 14:00:00    timeout          0.0
2        7 2020-01-04 13:57:59 2021-06-12 11:57:29  confirmed          1.0
3        7 2020-01-04 13:57:59 2021-06-13 12:58:28  confirmed          1.0
4        7 2020-01-04 13:57:59 2021-06-14 13:59:27  confirmed          1.0
5        2 2020-07-29 23:09:44 2021-01-22 00:00:00  confirmed          1.0
6        2 2020-07-29 23:09:44 2021-02-28 23:59:59    timeout          0.0
7        6 2020-12-09 10:39:37                 NaT        NaN          NaN
   user_id  confirmation_rate
0        2                0.5
1        3                0.0
2        6                0.0
3        7                1.0


In [ ]:
# LC 1193 - https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50

"""
T SQL

Q1
-- WITH tx AS (
--   SELECT *,
--     FORMAT(trans_date, 'yyyy-MM') AS month
--   FROM Transactions
-- )
-- SELECT
--   country,
--   month,
--   COUNT(*) AS trans_count,
--   SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
--   SUM(amount) AS trans_total_amount,
--   SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_total_amount
-- FROM tx
-- GROUP BY country, month;


Q2
SELECT 
    country,
    FORMAT(trans_date, 'yyyy-MM') AS month,
    COUNT(*) AS trans_count,
    SUM(IIF(state = 'approved', 1, 0)) AS approved_count,
    SUM(amount) AS trans_total_amount,
    SUM(IIF(state = 'approved', amount, 0)) AS approved_total_amount
FROM Transactions
GROUP BY country, FORMAT(trans_date, 'yyyy-MM')

the only difference is in Q2 I'm using format function twice but in 
Q1 the format is used once inside the cte. both gives same performance in LC.
also case and iif(inline if) are interchangeable
"""


data = [
    [121, 'US', 'approved', 1000, '2018-12-18'], 
    [122, 'US', 'declined', 2000, '2018-12-19'], 
    [123, 'US', 'approved', 2000, '2019-01-01'], 
    [124, 'DE', 'approved', 2000, '2019-01-07']
]
transactions_df = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})


def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    transactions['month'] = pd.to_datetime(transactions['trans_date']).dt.strftime('%Y-%m')
    transactions['state'] = transactions['state'].str.replace('approved', '1').replace('declined', '0')
    transactions['state'] = transactions['state'].astype(int)
    transactions['approved_total_amount'] = transactions['amount'] * transactions['state']
    resq = transactions.groupby(['month', 'country'], dropna=False).agg({
        'id' : 'count', 
        'state' : 'sum', 
        'amount' : 'sum', 
        'approved_total_amount' : 'sum'}).reset_index()
    return resq.rename(columns={'id' : 'trans_count', 'state' : 'approved_count', 'amount' : 'trans_total_amount'})


print(monthly_transactions(transactions_df))

    id country state  amount trans_date    month
0  121      US     1    1000 2018-12-18  2018-12
1  122      US     0    2000 2018-12-19  2018-12
2  123      US     1    2000 2019-01-01  2019-01
3  124      DE     1    2000 2019-01-07  2019-01
     month country  trans_count  approved_count  trans_total_amount  \
0  2018-12      US            2               1                3000   
1  2019-01      DE            1               1                2000   
2  2019-01      US            1               1                2000   

   approved_total_amount  
0                   1000  
1                   2000  
2                   2000  


In [ ]:
# LC 1174 - https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50


"""
WITH ranked_data AS (
    SELECT 
        *,
        RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS rnk
    FROM Delivery
)
SELECT
    ROUND(SUM(CASE 
        WHEN order_date = customer_pref_delivery_date THEN 1
        ELSE 0
    END) * 100.0 / COUNT(*), 2) AS immediate_percentage
FROM ranked_data
WHERE rnk = 1
"""


data = [
    [1, 1, '2019-08-01', '2019-08-02'], 
    [2, 2, '2019-08-02', '2019-08-02'], 
    [3, 1, '2019-08-11', '2019-08-12'], 
    [4, 3, '2019-08-24', '2019-08-24'], 
    [5, 3, '2019-08-21', '2019-08-22'], 
    [6, 2, '2019-08-11', '2019-08-13'], 
    [7, 4, '2019-08-09', '2019-08-09']
]
delivery_df = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({
    'delivery_id':'Int64', 
    'customer_id':'Int64', 
    'order_date':'datetime64[ns]', 
    'customer_pref_delivery_date':'datetime64[ns]'
})


def immediate_food_delivery(delivery: pd.DataFrame) -> pd.DataFrame:
    first = (
        (delivery.sort_values(['customer_id', 'order_date']))
        .drop_duplicates('customer_id', keep='first')
    )
    first['is_immediate'] = (first['order_date'] == first['customer_pref_delivery_date']).astype(int)
    immediate_mean = round(first['is_immediate'].mean() * 100, 2)
    return pd.DataFrame(data={'immediate_percentage': immediate_mean}, index=[0])

print(immediate_food_delivery(delivery_df))



   immediate_percentage
0                  50.0
